# Extracción de características HOG + LBP
Este notebook combina los descriptores HOG (Histogram of Oriented Gradients) y LBP (Local Binary Patterns) para crear un descriptor híbrido más robusto

In [1]:
import os
import glob
import numpy as np
import cv2
from skimage.feature import hog, local_binary_pattern
from tqdm import tqdm

# Rutas base
BASE_DIR = os.path.dirname(os.path.abspath('.'))
NORMALIZADAS_DIR = os.path.join(BASE_DIR, 'normalizacion', 'normalizadas')
HOG_LBP_DIR = os.path.join(BASE_DIR, 'HOG_LBP', 'hog_lbp_features')

# Parámetros HOG
hog_orientations = 9
hog_pixels_per_cell = (8, 8)
hog_cells_per_block = (2, 2)

# Parámetros LBP
lbp_radius = 3
lbp_n_points = 8 * lbp_radius
lbp_method = 'uniform'

print(f"Ruta de imágenes normalizadas: {NORMALIZADAS_DIR}")
print(f"Ruta de salida HOG+LBP: {HOG_LBP_DIR}")
print(f"\nParámetros HOG:")
print(f"- Orientaciones: {hog_orientations}")
print(f"- Píxeles por celda: {hog_pixels_per_cell}")
print(f"- Celdas por bloque: {hog_cells_per_block}")
print(f"\nParámetros LBP:")
print(f"- Radio: {lbp_radius}")
print(f"- Puntos: {lbp_n_points}")
print(f"- Método: {lbp_method}")

Ruta de imágenes normalizadas: /workspaces/fruit-detector-pricer/ProyectoFinal/normalizacion/normalizadas
Ruta de salida HOG+LBP: /workspaces/fruit-detector-pricer/ProyectoFinal/HOG_LBP/hog_lbp_features

Parámetros HOG:
- Orientaciones: 9
- Píxeles por celda: (8, 8)
- Celdas por bloque: (2, 2)

Parámetros LBP:
- Radio: 3
- Puntos: 24
- Método: uniform


In [2]:
def extraer_caracteristicas_hog_lbp(imagen):
    # Calcular HOG
    hog_features = hog(imagen, 
                       orientations=hog_orientations,
                       pixels_per_cell=hog_pixels_per_cell,
                       cells_per_block=hog_cells_per_block,
                       visualize=False)
    
    # Calcular LBP
    lbp = local_binary_pattern(imagen, 
                              P=lbp_n_points, 
                              R=lbp_radius, 
                              method=lbp_method)
    
    # Calcular histograma LBP
    n_bins = int(lbp.max() + 1)
    lbp_hist, _ = np.histogram(lbp.ravel(), 
                              bins=n_bins, 
                              range=(0, n_bins), 
                              density=True)
    
    # Concatenar características HOG y LBP
    caracteristicas_combinadas = np.concatenate([hog_features, lbp_hist])
    
    return caracteristicas_combinadas

In [3]:
def procesar_imagenes(origen_split):
    split_path = os.path.join(NORMALIZADAS_DIR, origen_split)
    if not os.path.exists(split_path):
        print(f"❌ Carpeta no encontrada: {split_path}")
        return
    
    # Crear directorio de salida si no existe
    out_split_path = os.path.join(HOG_LBP_DIR, origen_split)
    os.makedirs(out_split_path, exist_ok=True)
    
    # Obtener lista de clases (carpetas de frutas)
    clases = [d for d in os.listdir(split_path) 
             if os.path.isdir(os.path.join(split_path, d))]
    
    for clase in clases:
        # Crear directorio de salida para la clase
        clase_in_dir = os.path.join(split_path, clase)
        clase_out_dir = os.path.join(out_split_path, clase)
        os.makedirs(clase_out_dir, exist_ok=True)
        
        # Procesar cada imagen
        imagenes = glob.glob(os.path.join(clase_in_dir, '*_recorte.png'))
        
        for img_path in tqdm(imagenes, desc=f"Procesando {clase} en {origen_split}"):
            # Leer imagen
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"⚠️ No se pudo leer la imagen: {img_path}")
                continue
                
            # Extraer características HOG + LBP
            features = extraer_caracteristicas_hog_lbp(img)
            
            # Guardar características
            nombre_base = os.path.basename(img_path).replace('_recorte.png', '')
            out_path = os.path.join(clase_out_dir, f"{nombre_base}_hog_lbp.npy")
            np.save(out_path, features)

# Procesar imágenes de train y test
for split in ['train', 'test']:
    procesar_imagenes(split)

print("\n✓ Procesamiento HOG+LBP completado")
print("Las características combinadas han sido guardadas en la carpeta hog_lbp_features/")

Procesando Pineapple Mini 1 en test: 100%|██████████| 148/148 [00:01<00:00, 79.23it/s]


✓ Procesamiento HOG+LBP completado
Las características combinadas han sido guardadas en la carpeta hog_lbp_features/
